In [2]:
# # Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from EDA_script import *
# # Set options
# pd.options.display.max_rows = 999
# pd.options.display.max_columns = 999

# train_x_raw = pd.read_csv("../01-Data/X_train.csv", low_memory = True, index_col=0)
# train_y_raw = pd.read_csv("../01-Data/y_train.csv", low_memory = True, index_col=0)
# test_x_raw = pd.read_csv("../01-Data/X_test.csv", low_memory=True, index_col=0)

# df_train = pd.DataFrame(train_x_raw)
# df_test = pd.DataFrame(test_x_raw)
# df_y = pd.DataFrame(train_y_raw)

v228b
v231b
v233b
v251b
v275b_N2
v275b_N1
v281a
[('country', 'v275c_N2'), ('country', 'v275c_N1'), ('v20a', 'v20b'), ('v30b', 'v30c'), ('v45b', 'v45c'), ('v96a', 'v96b'), ('v135_11c', 'v136_11c'), ('v135_11c', 'v138_11c'), ('v135_11c', 'v141_11c'), ('v136_11c', 'v141_11c'), ('v138_11c', 'v141_11c'), ('v176_DK', 'v177_DK'), ('v176_DK', 'v178_DK'), ('v176_DK', 'v179_DK'), ('v176_DK', 'v180_DK'), ('v176_DK', 'v181_DK'), ('v176_DK', 'v182_DK'), ('v176_DK', 'v183_DK'), ('v177_DK', 'v178_DK'), ('v177_DK', 'v179_DK'), ('v177_DK', 'v180_DK'), ('v177_DK', 'v181_DK'), ('v177_DK', 'v182_DK'), ('v177_DK', 'v183_DK'), ('v178_DK', 'v179_DK'), ('v178_DK', 'v180_DK'), ('v178_DK', 'v181_DK'), ('v178_DK', 'v182_DK'), ('v178_DK', 'v183_DK'), ('v179_DK', 'v180_DK'), ('v179_DK', 'v181_DK'), ('v179_DK', 'v182_DK'), ('v179_DK', 'v183_DK'), ('v180_DK', 'v181_DK'), ('v180_DK', 'v182_DK'), ('v180_DK', 'v183_DK'), ('v181_DK', 'v182_DK'), ('v181_DK', 'v183_DK'), ('v182_DK', 'v183_DK'), ('v221_DK', 'v222_DK'), ('v

# Simple Model Run
##  xgBoost model set up

In [3]:
from sklearn.metrics import log_loss

label_mapping = {-1: 0, 1: 1, 2: 2, 3: 3, 4: 4}
df_y = df_y.replace(label_mapping)


# dtrain = xgb.DMatrix(df_train, label=df_y, enable_categorical=True)
# dtest = xgb.DMatrix(df_test, enable_categorical=True)


import cupy as cp

# Convert features and labels to Cupy arrays
df_train_gpu = cp.asnumpy(df_train)
df_y_gpu = cp.asnumpy(df_y)

# Create DMatrix from Cupy arrays
dtrain = xgb.DMatrix(df_train_gpu, label=df_y_gpu, enable_categorical=True)

# Convert test data to Cupy arrays
df_test_gpu = cp.asnumpy(df_test)

# Create DMatrix for test data
dtest = xgb.DMatrix(df_test_gpu, enable_categorical=True)


params = {
    'max_depth': 8,
    'eta': 0.1,
    'objective': 'multi:softprob',
    'num_class': 5,
    'eval_metric': 'mlogloss',
    'device' : 'cuda',
    'predictor': 'gpu_predictor'
}



num_boost_round = 1000

## Xgboost Model GridSearchCV

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

# Assume df_train, df_test, and df_y are already defined and preprocessed
# as per your setup

    # parmas = {
    #     'colsample_bytree': 0.6280261676059677,
    #     'eta': 0.06360779210240283,
    #     'gamma': 0.9695846277645586,
    #     'max_depth': 8.425929763527801,
    #     'min_child_weight': 5.697494707820946,
    #     'reg_alpha': 0.8948273504276488,
    #     'reg_lambda': 3.3915999152443406,
    #     'subsample': 0.8531245410138701
    # }

# Define the parameter grid to search
param_grid = {
    'device' : ['cuda'],
    'max_depth': (7,9),
    'learning_rate': (0.05, 0.1),
    'n_estimators': (500, 1000),
    'subsample': (0,7, 0.8),
    'gamma': (1, 3),
    'colsample_bytree': (0.4, 0.7),
}

# Instantiate the XGBClassifier (note: use objective 'multi:softprob' for multi-class)
xgb_model = XGBClassifier(objective='multi:softprob', num_class=5, eval_metric='mlogloss', use_label_encoder=False,device='cuda')

# Set up GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_log_loss', n_jobs=-1, cv=5, verbose=2)

# Fit the GridSearchCV object to find the best parameters
grid_search.fit(df_train_gpu, df_y_gpu)

# Best parameter set found
print("Best parameters found: ", grid_search.best_params_)

# Best score
print("Best score (neg_log_loss): ", grid_search.best_score_)

# You can also use the best estimator directly to make predictions
# best_estimator = grid_search.best_estimator


NameError: name 'df_train_gpu' is not defined

## Cross Validation

In [ ]:
from xgboost import cv

# params and num_boost_round provided above
xgb_cv = cv(dtrain=dtrain, params=params, nfold=10,
            num_boost_round=num_boost_round, early_stopping_rounds=5,
            metrics="mlogloss", as_pandas=True, seed=123)

xgb_cv

## Xgboost Training

In [ ]:
evals_result = {}
bst = xgb.train(params, dtrain, num_boost_round, 
                evals=[(dtrain, 'train')], evals_result=evals_result, 
                verbose_eval=False)
print(f"Training Multiclass Logarithmic Loss: {evals_result['train']['mlogloss'][-1]}")

y_test_probs = bst.predict(dtest)

class_order = [0, 1, 2, 3, 4]
class_mapping = {class_label: f"Class_{class_label}" for class_label in class_order}

y_train_probs = bst.predict(dtrain)
val_log_loss = log_loss(df_y, y_train_probs, labels=class_order)
print(f"Validation Multiclass Logarithmic Loss: {val_log_loss}")

## Generate Submission csv

In [ ]:
submission_df = pd.DataFrame(y_test_probs, columns=class_mapping.values())
submission_df.columns = ['no answer', 'very important', 'quite important', 'not important', 'not at all important']
submission_df.insert(0, 'id', df_test.index)

# Save the submission file
# submission_file = ('submission.csv')
# submission_df.to_csv(submission_file, index=False)